# Biostat 257 Homework 2

**Due Apr 30 @ 11:59PM**

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.6.0)
  CPU: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)


Consider a linear mixed effects model
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effect predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effect predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

## Q1 Formula (10 pts)

Write down the log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ given parameters $(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$. 

**Hint:** For non-statisticians, feel free to ask for help in class or office hours. Point of this exercise is computing not statistics.

$$
\mathbf{Y}_i \sim N(\mathbf{X}_i \boldsymbol{\beta}, \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \boldsymbol{\Sigma}_{q \times q} \mathbf{Z}_i^T)
$$

$$
\ell = - \frac{n_i}{2} \log 2\pi - 
\frac{1}{2} \log \text{det}
(\sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \boldsymbol{\Sigma}_{q \times q} \mathbf{Z}_i^T) - 
\frac{1}{2} (\boldsymbol{y}_i - \mathbf{X}_i \boldsymbol{\beta})^T  
(\sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \boldsymbol{\Sigma}_{q \times q} \mathbf{Z}_i^T)^{-1} (\boldsymbol{y}_i - \mathbf{X}_i \boldsymbol{\beta})
$$

If we let $\boldsymbol{\Omega} = \mathbf{I}_{n_i} + \frac{1}{\sigma^2}\mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T = \mathbf{I}_{n_i} + \frac{1}{\sigma^2}\mathbf{Z}_i \mathbf{L} \mathbf{L}^T \mathbf{Z}_i^T$, $\mathbf{M} = \mathbf{I}_{q} + \frac{1}{\sigma^2} \mathbf{L}^T \mathbf{Z}_i^T \mathbf{Z}_i \mathbf{L}$, and $\boldsymbol{r}_i = \boldsymbol{y}_i - \mathbf{X}_i \boldsymbol{\beta}$,

$$
\text{det}(\boldsymbol{\Omega}) = (\sigma^2)^{n_i} \text{det}(\mathbf{M})
$$

$$
\boldsymbol{\Omega}^{-1} = \mathbf{I}_{n_i} - \frac{1}{\sigma^2} \mathbf{Z}_i \mathbf{L} \mathbf{M}^{-1} \mathbf{L}^T \mathbf{Z}_i^T
$$

$$
\therefore \ell = - \frac{n_i}{2} \log 2\pi - 
\frac{n_i}{2} \log \sigma^2 -
\frac{1}{2} \log \text{det}(\mathbf{M}) - 
\frac{1}{2\sigma^2} \boldsymbol{r}_i^T  
(\mathbf{I}_{n_i} - \frac{1}{\sigma^2} \mathbf{Z}_i \mathbf{L} \mathbf{M}^{-1} \mathbf{L}^T \mathbf{Z}_i^T)
\boldsymbol{r}_i
$$

## Q2 Start-up code

Use the following template to define a type `LmmObs` that holds an LMM datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$. 

In [2]:
# define a type that holds LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y :: Vector{T}
    X :: Matrix{T}
    Z :: Matrix{T}
    # working arrays
    # whatever intermediate vectors/arrays you may want to pre-allocate
    XᵗXβ       :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    M          :: Matrix{T}
    yty        :: Float64
    xty        :: Vector{T}
    zty        :: Vector{T}
    LᵗZᵗr      :: Vector{T}
end

# constructor
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
        ) where T <: AbstractFloat
    XᵗXβ       = Vector{T}(undef, size(X, 2))
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    M          = similar(ztz)
    yty        = transpose(y) * y 
    xty        = transpose(X) * y
    zty        = transpose(Z) * y
    LᵗZᵗr      = Vector{T}(undef, size(Z, 2))
    LmmObs(y, X, Z, XᵗXβ, xtx, ztx, ztz, M, yty, xty, zty, LᵗZᵗr)
end

LmmObs

Write a function, with interface   
```julia
logl!(obs, β, L, σ²)
```
that evaluates the log-likelihood of the $i$-th datum. Here `L` is the lower triangular Cholesky factor from the Cholesky decomposition `Σ=LL'`. Make your code efficient in the $n_i \gg q$ case. Think the intensive longitudinal measurement setting.  

In [3]:
function logl!(
        obs :: LmmObs{T}, 
        β   :: Vector{T}, 
        L   :: Matrix{T}, 
        σ²  :: T) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)    
    
    # compute M = I + LᵗZᵗZL / σ²
    copyto!(obs.M, obs.ztz)
    BLAS.trmm!('R', 'L', 'N', 'N', 1.0, L, obs.M)
    BLAS.trmm!('L', 'L', 'T', 'N', 1.0, L, obs.M)
    obs.M .= obs.M ./ σ²
    for i in 1:q
        obs.M[i, i] += 1.0
    end
    LAPACK.potrf!('U', obs.M)  
    
    # compute determinant
    detM = n / 2 * log(σ²) + logdet(UpperTriangular(obs.M))
    
    # compute rᵗr = yᵗy - 2βᵗXᵗy + βᵗXᵗXβ
    BLAS.gemv!('N', 1.0, obs.xtx, β, 0.0, obs.XᵗXβ)
    rᵗr = obs.yty - 2 * dot(obs.xty, β) + dot(obs.XᵗXβ, β)
    
    # compute rᵗZLM⁻¹LᵗZᵗr
    copyto!(obs.LᵗZᵗr, obs.zty)
    BLAS.gemv!('N', -1.0, obs.ztx, β, 1.0, obs.LᵗZᵗr)
    BLAS.trmv!('L', 'T', 'N', L, obs.LᵗZᵗr)
    BLAS.trsv!('U', 'T', 'N', obs.M, obs.LᵗZᵗr)
    rᵗZLM⁻¹LᵗZᵗr = dot(obs.LᵗZᵗr, obs.LᵗZᵗr)

    # compute log-likelihood
    loglike = - n / 2 * log(2π) - detM - 1 / (2 * σ²) * rᵗr +
        1 / (2 * σ²^2) * rᵗZLM⁻¹LᵗZᵗr
    
    return loglike
end

logl! (generic function with 1 method)

### Note on several ways to increase efficiency:
+ We can decrease computation and the number of FLOPS by operating at the level of $p$ and $q$, not $n_i$. Along the same lines, we can pre-calculate sufficient statistics. We can also leverage the structure inherent in matrix (e.g. `LowerTriangular`), taking full advantage of multiple dispatch in Julia.
+ We can decrease memory mapping by pre-allocation.
+ BLAS seemed in general faster than inplace functions, so when there are multiple ways of writing the same function as below, we can go with BLAS.

```
BLAS.trmm!('R', 'L', 'N', 'N', 1.0, L, obs.M) ≈ rmul!(obs.M, LowerTriangular(L))
BLAS.trmm!('L', 'L', 'T', 'N', 1.0, L, obs.M) ≈ lmul!(transpose(LowerTriangular(L)), obs.M)
LAPACK.potrf!('U', obs.M) ≈ cholesky!(Symmetric(obs.M))
BLAS.trmv!('L', 'T', 'N', L, obs.LᵗZᵗr) ≈ lmul!(transpose(LowerTriangular(L)), obs.LᵗZᵗr)
BLAS.gemv!('N', 1.0, obs.xtx, β, 0.0, obs.XᵗXβ) ≈ mul!(obs.XᵗXβ, obs.xtx, β)
BLAS.gemv!('N', -1.0, obs.ztx, β, 1.0, obs.LᵗZᵗr) ≈ mul!(obs.LᵗZᵗr, obs.ztx, β, 1.0, -1.0)
BLAS.trsv!('U', 'T', 'N', obs.M, obs.LᵗZᵗr) ≈ ldiv!(transpose(UpperTriangular(obs.M)), obs.LᵗZᵗr)
```

**Hint**: This function shouldn't be very long. Mine, obeying 80-character rule, is 25 lines. If you find yourself writing very long code, you're on the wrong track. Think about algorithm first then use BLAS functions to reduce memory allocations.

## Q3 Correctness (15 pts)

Compare your result (both accuracy and timing) to the [Distributions.jl](https://juliastats.org/Distributions.jl/stable/multivariate/#Distributions.AbstractMvNormal) package using following data.

In [4]:
using BenchmarkTools, Distributions, LinearAlgebra, Random

Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)
# form an LmmObs object
obs = LmmObs(y, X, Z);

This is the standard way to evaluate log-density of a multivariate normal, using the Distributions.jl package. Let's evaluate the log-likelihood of this datum.

In [5]:
μ  = X * β
Ω  = Z * Σ * transpose(Z) +  σ² * I
mvn = MvNormal(μ, Symmetric(Ω)) # MVN(μ, Σ)
logpdf(mvn, y)

-3261.917755918759

Check that your answer matches that from Distributions.jl

In [6]:
L = Matrix(cholesky(Σ).L)
logl!(obs, β, L, σ²)

-3261.9177559187606

**You will lose all 15 + 30 + 30 = 75 points** if the following statement throws `AssertionError`.

In [7]:
@assert logl!(obs, β, Matrix(cholesky(Σ).L), σ²) ≈ logpdf(mvn, y)

## Q4 Efficiency (30 pts)

Benchmarking your code and compare to the Distributions.jl function `logpdf`.

In [8]:
# benchmark the `logpdf` function in Distribution.jl
bm1 = @benchmark logpdf($mvn, $y)

BenchmarkTools.Trial: 
  memory estimate:  30.55 MiB
  allocs estimate:  5
  --------------
  minimum time:     8.832 ms (0.00% GC)
  median time:      10.458 ms (0.00% GC)
  mean time:        11.092 ms (14.86% GC)
  maximum time:     15.937 ms (34.27% GC)
  --------------
  samples:          450
  evals/sample:     1

In [9]:
# benchmark your implementation
L = Matrix(cholesky(Σ).L)
bm2 = @benchmark logl!($obs, $β, $L, $σ²)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     643.272 ns (0.00% GC)
  median time:      668.036 ns (0.00% GC)
  mean time:        732.591 ns (0.00% GC)
  maximum time:     2.531 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     169

The points you will get is
$$
\frac{x}{10000} \times 30,
$$
where $x$ is the speedup of your program against the standard method.

In [10]:
# this is the points you'll get
clamp(median(bm1).time / median(bm2).time / 10_000 * 30, 0, 30)

30.0

**Hint**: Apparently I am using 10000 as denominator because I expect your code to be at least $10000 \times$ faster than the standard method.

## Q5 Memory (30 pts)

You want to avoid memory allocation in the "hot" function `logl!`. You will lose 1 point for each `1 KiB = 1024 bytes` memory allocation. In other words, the points you get for this question is

In [11]:
clamp(30 - median(bm2).memory / 1024, 0, 30)

30.0

**Hint**: I am able to reduce the memory allocation to 0 bytes.

## Q6 Misc (15 pts)

Coding style, Git workflow, etc. For reproducibity, make sure we (TA and myself) can run your Jupyter Notebook. That is how we grade Q4 and Q5. If we cannot run it, you will get zero points.